In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
!git clone -b v1.2 https://github.com/camenduru/text-generation-webui.git
%cd /content/text-generation-webui
!pip install -r requirements.txt 

In [ ]:
!mkdir /content/text-generation-webui/repositories
%cd /content/text-generation-webui/repositories
!git clone -b v1.2 https://github.com/camenduru/GPTQ-for-LLaMa.git
%cd GPTQ-for-LLaMa
!python setup_cuda.py install

drive_folder = '/content/drive/MyDrive/text-generation-webui'
if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)
!mv /content/text-generation-webui/* "$drive_folder/"

In [ ]:
!apt-get -y install -qq aria2

In [ ]:
import subprocess

def download_file(url, path, filename):
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {path} -o {filename}"
    print(f"Downloading {filename}...")
    subprocess.run(command, shell=True)
    print(f"{filename} downloaded successfully.")
  
model = "vicuna"  # Set this to "alpaca" to select the alpaca model
if model == "alpaca":
    base_url = "https://huggingface.co/anon8231489123/gpt4-x-alpaca-13b-native-4bit-128g"
    safetensors = "4bit-128g.safetensors"
elif model == "vicuna":
    base_url = "https://huggingface.co/anon8231489123/vicuna-13b-GPTQ-4bit-128g"
    safetensors = "vicuna-13b-4bit-128g.safetensors"
else:
    raise ValueError("Invalid model selected")

base_path = "/content/drive/MyDrive/text-generation-webui/models/gpt4-x-" + model + "-13b-native-4bit-128g"

download_file(f"{base_url}/raw/main/config.json", base_path, "config.json")
download_file(f"{base_url}/raw/main/generation_config.json", base_path, "generation_config.json")
download_file(f"{base_url}/raw/main/special_tokens_map.json", base_path, "special_tokens_map.json")
download_file(f"{base_url}/resolve/main/tokenizer.model", base_path, "tokenizer.model")
download_file(f"{base_url}/raw/main/tokenizer_config.json", base_path, "tokenizer_config.json")
download_file(f"{base_url}/resolve/main/{safetensors}", base_path, safetensors)

In [ ]:
import os  

%cd /content/drive/MyDrive/text-generation-webui

auth_token_path = "/content/drive/MyDrive/text-generation-webui/auth_token.txt"

# Set your username and password here (or use the default)
user = "username" 
password = "password"
 
auth = input("Do you want to enable authentication? (yes/no): ") 

if auth.lower() == "yes":
    with open(auth_token_path, "w") as f:
        f.write(f"{user}:{password}")
    print("auth_token.txt created successfully.") 

if os.path.exists(auth_token_path):
    !python server.py --share --chat --wbits 4 --groupsize 128 --gradio-auth-path {auth_token_path}
else:
    !python server.py --share --chat --wbits 4 --groupsize 128